In [4]:
import duckdb

In [1]:
%pwd

'/Users/maxwellpatterson/Desktop/classes/spring/eds213/eds213-materials/week3'

In [2]:
%ls

01_ASDN_Readme.txt        build-database*           site.csv
ASDN_Bird_eggs.csv        class-notes-wk6.ipynb     species.csv
ASDN_Bird_nests.csv       database.db               species_fixed.csv
ASDN_Camp_assignment.csv  personnel.csv             theloop.sh
addition.sh               schema-build-script.sql
asdn-er-diagram.png       scripts.sql


1. Create a a connection and a cursor

In [5]:
conn = duckdb.connect("database.db")

In [6]:
cur = conn.cursor()

In [6]:
cur.execute("SELECT * FROM Site LIMIT 5;")

Now we want results. There are three ways of getting them

In [7]:
cur.fetchall()

[('barr',
  'Barrow',
  'Alaska, USA',
  71.30000305175781,
  -156.60000610351562,
  220.39999389648438),
 ('burn',
  'Burntpoint Creek',
  'Ontario, Canada',
  55.20000076293945,
  -84.30000305175781,
  63.0),
 ('bylo',
  'Bylot Island',
  'Nunavut, Canada',
  73.19999694824219,
  -80.0,
  723.5999755859375),
 ('cakr',
  'Cape Krusenstern',
  'Alaska, USA',
  67.0999984741211,
  -163.5,
  54.099998474121094),
 ('cari',
  'Canning River Delta',
  'Alaska, USA',
  70.0999984741211,
  -145.8000030517578,
  722.0)]

Cursors don't store anything, they just transfer queries to the database and get results back.

In [8]:
cur.fetchall()

[]

In [9]:
cur.execute("SELECT Nest_ID FROM Bird_nests LIMIT 10")

In [10]:
cur.fetchall()

[('14HPE1',),
 ('11eaba',),
 ('11eabaagc01',),
 ('11eabaagv01',),
 ('11eababbc02',),
 ('11eababsv01',),
 ('11eabaduh01',),
 ('11eabaduv01',),
 ('11eabarpc01',),
 ('11eabarpc02',)]

In [11]:
cur.execute("SELECT Nest_ID FROM Bird_nests LIMIT 10")
[t[0] for t in cur.fetchall()]

['14HPE1',
 '11eaba',
 '11eabaagc01',
 '11eabaagv01',
 '11eababbc02',
 '11eababsv01',
 '11eabaduh01',
 '11eabaduv01',
 '11eabarpc01',
 '11eabarpc02']

2. Get the one result, or the next result

In [13]:
cur.execute("SELECT COUNT(*) FROM Bird_nests")
cur.fetchall()

[(1547,)]

In [14]:
cur.execute("SELECT COUNT(*) FROM Bird_nests")
cur.fetchone()

(1547,)

In [15]:
cur.execute("SELECT COUNT(*) FROM Bird_nests")
cur.fetchone()[0]

1547

Using an iterator -- but DuckDB doesn't support iterators

In [16]:
cur.execute("SELECT Nest_ID FROM Bird_nests LIMIT 10")
for row in cur:
    print(f"got {row[0]}")

TypeError: 'duckdb.duckdb.DuckDBPyConnection' object is not iterable

A workaround:

In [17]:
cur.execute("SELECT Nest_ID FROM Bird_nests LIMIT 10")
while True:
    row = cur.fetchone()
    if row == None:
        break
    # do something with row
    print(f"got nest ID {row[0]}")

got nest ID 14HPE1
got nest ID 11eaba
got nest ID 11eabaagc01
got nest ID 11eabaagv01
got nest ID 11eababbc02
got nest ID 11eababsv01
got nest ID 11eabaduh01
got nest ID 11eabaduv01
got nest ID 11eabarpc01
got nest ID 11eabarpc02


In [20]:
cur.execute("""
            CREATE TEMP TABLE temp_table AS
            SELECT * FROM Bird_nests LIMIT 10
            """)

CatalogException: Catalog Error: Table with name "temp_table" already exists!

In [21]:
cur.execute("SELECT * FROM temp_table")

In [22]:
cur.fetchall()

[('b14.6',
  2014,
  'chur',
  '14HPE1',
  'sepl',
  'vloverti',
  datetime.date(2014, 6, 14),
  None,
  3,
  None,
  None),
 ('b11.7',
  2011,
  'eaba',
  '11eaba',
  'wrsa',
  'bhill',
  datetime.date(2011, 7, 10),
  'searcher',
  4,
  None,
  None),
 ('b11.6',
  2011,
  'eaba',
  '11eabaagc01',
  'amgp',
  'dkessler',
  datetime.date(2011, 6, 24),
  'searcher',
  4,
  6.0,
  'float'),
 ('b11.6',
  2011,
  'eaba',
  '11eabaagv01',
  'amgp',
  'dkessler',
  datetime.date(2011, 6, 25),
  'searcher',
  3,
  3.0,
  'float'),
 ('b11.6',
  2011,
  'eaba',
  '11eababbc02',
  'bbpl',
  'dkessler',
  datetime.date(2011, 6, 24),
  'searcher',
  4,
  4.0,
  'float'),
 ('b11.7',
  2011,
  'eaba',
  '11eababsv01',
  'wrsa',
  'bhill',
  datetime.date(2011, 7, 7),
  'searcher',
  4,
  2.0,
  'float'),
 ('b11.6',
  2011,
  'eaba',
  '11eabaduh01',
  'dunl',
  'dkessler',
  datetime.date(2011, 6, 28),
  'searcher',
  3,
  2.0,
  'float'),
 ('b11.6',
  2011,
  'eaba',
  '11eabaduv01',
  'dunl',
  'dk

A note on fragility

For example:
INSERT INTO Site VALUES ("abcd", "Foo", 35.7, -119.5, "?")

A less fragile way of expressing the same thing:
INSERT INTO Site (Code, Site_name, Latitude, Longitude, Something_else)
    VALUES ("abcd", "Foo", 35.7, -119.5, "?")



In [23]:
cur.execute("SELECT * FROM Site LIMIT 3")
cur.fetchall()

[('barr',
  'Barrow',
  'Alaska, USA',
  71.30000305175781,
  -156.60000610351562,
  220.39999389648438),
 ('burn',
  'Burntpoint Creek',
  'Ontario, Canada',
  55.20000076293945,
  -84.30000305175781,
  63.0),
 ('bylo',
  'Bylot Island',
  'Nunavut, Canada',
  73.19999694824219,
  -80.0,
  723.5999755859375)]

In [25]:
cur.execute("SELECT Site_name, Code, Latitude, Longitude FROM Site LIMIT 3")
cur.fetchall()

[('Barrow', 'barr', 71.30000305175781, -156.60000610351562),
 ('Burntpoint Creek', 'burn', 55.20000076293945, -84.30000305175781),
 ('Bylot Island', 'bylo', 73.19999694824219, -80.0)]

An extended exampke: How many nests do we have for each species?

Approach: first get all species. Then execute a count query for each species.

In [30]:
query = """
    SELECT COUNT(*) FROM Bird_nests
    WHERE Species = '%s'
    """

cur.execute("SELECT Code FROM Species LIMIT 3")
for row in cur.fetchall(): #DuckDB
    code = row[0]
    prepared_query = query % code
    #print(prepared_query)
    cur2 = conn.cursor()
    cur2.execute(prepared_query)
    print(f"Species {code} has {cur2.fetchone()[0]} nests")
    cur2.close()

Species agsq has 0 nests
Species amcr has 0 nests
Species amgp has 29 nests


The above Python interpolation is dangerous and has caused my database hacks. Theres a better way

In [33]:
query = """
    SELECT COUNT(*) FROM Bird_nests
    WHERE Species = ?
    """

cur.execute("SELECT Code FROM Species LIMIT 3")
for row in cur.fetchall(): #DuckDB
    code = row[0]
    # prepared_query = query % code
    #print(prepared_query)
    cur2 = conn.cursor()
    cur2.execute(query, [code])
    print(f"Species {code} has {cur2.fetchone()[0]} nests")
    cur2.close()

Species agsq has 0 nests
Species amcr has 0 nests
Species amgp has 29 nests


Let's illustrate the danger with a different example

In [35]:
abbrev = "TS"
name = "Taylor Swift"
cur.execute("""
            INSERT INTO Personnel (Abbreviation, Name)
            VALUES ('%s', '%s')
            """ % (abbrev, name)
            )

In [36]:
abbrev = "CO"
name = "Conan O'Brien"
cur.execute("""
            INSERT INTO Personnel (Abbreviation, Name)
            VALUES ('%s', '%s')
            """ % (abbrev, name)
            )

ParserException: Parser Error: syntax error at or near "Brien"

In [39]:
abbrev = "CO"
name = "Conan O'Brien"
cur.execute("""
            INSERT INTO Personnel (Abbreviation, Name)
            VALUES (?, ?)
            """ 
            ,
            [abbrev, name])

In [41]:
cur.execute("SELECT * FROM Personnel")
cur.fetchall()[-3:]

[('mcorrell', 'Maureen Correll'),
 ('TS', 'Taylor Swift'),
 ('CO', "Conan O'Brien")]

In [7]:
inner_query = """
SELECT COUNT(*) AS num_nests
FROM Bird_nests
WHERE Observer = ?
"""

outer_query = """
SELECT DISTINCT Observer FROM Bird_nests"""

for row in cur.execute(outer_query).fetchall():
    observer = row[0]
    cur2 = conn.cursor()
    cur2.execute(inner_query, [observer])
    print(f"Observer {observer} gathered {cur2.fetchone()[0]} nests")

Observer vloverti gathered 54 nests
Observer rlanctot gathered 40 nests
Observer abankert gathered 17 nests
Observer bhill gathered 55 nests
Observer ssaalfeld gathered 13 nests
Observer wenglish gathered 18 nests
Observer lworing gathered 14 nests
Observer mballvanzee gathered 2 nests
Observer dkessler gathered 69 nests
Observer bharrington gathered 245 nests
Observer lmckinnon gathered 249 nests
Observer dhodkinson gathered 15 nests
Observer jzamuido gathered 11 nests
Observer amould gathered 42 nests
Observer bkaselow gathered 4 nests
Observer jflamarre gathered 43 nests
Observer edastrous gathered 38 nests
Observer mbwunder gathered 4 nests
Observer None gathered 0 nests
Observer kkalasz gathered 12 nests


In [8]:
import pandas as pd

In [9]:
df = pd.read_sql("SELECT * FROM Site", conn)

<ipython-input-9-36a7b332205d>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM Site", conn)
